In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
books=pd.read_csv("Books.csv",low_memory=False)
Ratings=pd.read_csv("Ratings.csv")
Users = pd.read_csv("Users.csv")

In [12]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [13]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [14]:
books=books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [15]:
books.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


In [16]:
books=books.rename(columns={
    "Book-Title" : "title",
    "Book-Author":"author",
    "Year-Of-Publication":"year",
    "Publisher":"publisher",
    "Image-URL-L":"img_url",
})

In [17]:
books.head(1)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


In [18]:
Users.head(1)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [19]:
Users.shape

(278858, 3)

In [20]:
Ratings.head(1)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [21]:
Ratings.shape

(1149780, 3)

In [22]:
print(books.shape)
print(Users.shape)
print(Ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [23]:
Ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [24]:
Ratings.rename(columns={
    "User-ID" :"user_id",
    "Book-Rating":"rating"
},inplace=True)

In [25]:
Ratings.head(1)

,user_id,ISBN,rating
0,276725,034545104X,0


In [26]:
Ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [27]:
Ratings['user_id'].unique().shape

(105283,)

In [28]:
x = Ratings['user_id'].value_counts() > 200

In [29]:
x[x].shape

(899,)

In [30]:
y = x[x].index

In [31]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [32]:
ratings=Ratings[Ratings['user_id'].isin(y)]

In [33]:
ratings.shape

(526356, 3)

In [34]:
books.head(1)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


In [35]:
ratings_with_books= ratings.merge(books, on="ISBN")

In [36]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [37]:
ratings_with_books.shape

(487671, 8)

In [38]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [39]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [40]:
num_rating.rename(columns={
    "rating":"num_of_rating"
},inplace=True)

In [41]:
num_rating.head(1)

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2


In [42]:
final_rating=ratings_with_books.merge(num_rating, on="title")

In [43]:
final_rating.head(1)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [44]:
final_rating.shape

(487671, 9)

In [45]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [46]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [47]:
final_rating.shape

(61853, 9)

In [48]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [49]:
final_rating.shape

(59850, 9)

In [50]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [51]:
book_pivot=final_rating.pivot_table(columns="user_id",index="title",values="rating")

In [52]:
book_pivot.head(1)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [53]:
book_pivot.shape

(742, 888)

In [54]:
book_pivot.fillna(0,inplace=True)

In [55]:
book_pivot.head(2)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
from scipy.sparse import csr_matrix

In [57]:
book_sparse= csr_matrix(book_pivot)

In [59]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm="brute")

In [60]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [61]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [62]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [63]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [64]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [65]:
books_name = book_pivot.index

In [66]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [68]:
import pickle
pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(books_name, open('artifacts/book_name.pkl','wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl','wb'))

In [69]:
# Testing the recommendation
def recommend_book(book_name):
  book_id=np.where(book_pivot.index==book_name)[0][0]
  distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  for i in range(len(suggestion)):
    books=book_pivot.index[suggestion[i]]
    for j in books:
      print(j)

In [70]:
recommend_book("1984")
  

1984
No Safe Place
A Civil Action
Foucault's Pendulum
Long After Midnight
Abduction
